In [55]:
!pip install pennylane

In [ ]:
# === IMPORTACIONES ===
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import QNGOptimizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import zipfile
import matplotlib.pyplot as plt

# === CONFIGURACIÓN DEL DISPOSITIVO ===
num_qubits = 4
num_layers = 4
dev = qml.device("default.qubit", wires=num_qubits)

# === CIRCUITOS ===
def stateprep(x):
    qml.BasisEmbedding(x, wires=range(num_qubits))

@qml.qnode(dev, interface="autograd")
def qnode_cost(param_pack):
    weights, x = param_pack
    stateprep(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface="autograd")
def qnode_only_weights(weights):
    # Usa global_x_batch[0] solo como dummy para el cálculo del tensor métrico
    stateprep(global_x_batch[0])
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

def classifier(weights, bias, x):
    return qnode_cost((weights, x)) + bias

# === DATASET TITANIC ===
with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")

df = pd.read_csv("titanic_data/train.csv")
df['Pclass'] = df['Pclass'].astype(str)
df = pd.concat([df, pd.get_dummies(df[['Pclass', 'Sex', 'Embarked']])], axis=1)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['is_child'] = df['Age'].map(lambda x: 1 if x < 12 else 0)

cols = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']
X = df[cols].astype(int).values
Y = (df['Survived'] * 2 - 1).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = np.array(X_train, requires_grad=False, dtype=int)
X_test = np.array(X_test, requires_grad=False, dtype=int)
Y_train = np.array(Y_train, requires_grad=False)
Y_test = np.array(Y_test, requires_grad=False)

# === PARÁMETROS DEL MODELO ===
shape = qml.StronglyEntanglingLayers.shape(num_layers, num_qubits)
weights = 0.1 * np.random.randn(*shape)
weights.requires_grad = True
bias = np.tensor(0.0, requires_grad=True)

qng_opt = QNGOptimizer(stepsize=0.1)
bias_lr = 0.01

# === ENTRENAMIENTO CON MINI-BATCH ===
batch_size = 16
num_epochs = 100
log_interval = 5

acc_hist = []
bias_hist = []

print("== INICIO DEL ENTRENAMIENTO ==")

for it in range(num_epochs):
    idx = np.random.choice(len(X_train), batch_size, replace=False)
    global_x_batch = X_train[idx]
    y_batch = Y_train[idx]

    # Paso 1: actualizar pesos (usamos solo la primera muestra para adjoint)
    metric_tensor = qml.adjoint_metric_tensor(qnode_only_weights)
    weights = qng_opt.step(qnode_only_weights, weights, metric_tensor_fn=metric_tensor)
    weights.requires_grad = True

    # Paso 2: actualizar bias usando promedio del gradiente en el batch
    preds = np.array([classifier(weights, bias, xi) for xi in global_x_batch])
    grads = -2 * (y_batch - preds)
    avg_grad = np.mean(grads)
    bias = bias - bias_lr * avg_grad

    # Registro
    if it % log_interval == 0 or it == num_epochs - 1:
        preds_train = [np.sign(classifier(weights, bias, xi)) for xi in X_train]
        acc = accuracy_score(Y_train, preds_train)
        acc_hist.append(acc)
        bias_hist.append(bias.item())
        print(f"Iter {it:3d} | Accuracy: {acc:.3f} | Bias: {bias:.3f}")

# === EVALUACIÓN FINAL ===
print("\n== EVALUACIÓN FINAL ==")
preds_test = [np.sign(classifier(weights, bias, x)) for x in X_test]
acc_test = accuracy_score(Y_test, preds_test)
print(f"Test accuracy: {acc_test:.3f}")

# === VISUALIZACIÓN ===
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, num_epochs, log_interval), acc_hist, label="Train Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training Accuracy")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(np.arange(0, num_epochs, log_interval), bias_hist, label="Bias")
plt.xlabel("Epoch")
plt.ylabel("Bias")
plt.title("Bias Evolution")
plt.grid(True)

plt.tight_layout()
plt.show()


== INICIO DEL ENTRENAMIENTO ==
Iter   0 | Accuracy: 0.782 | Bias: -0.002
Iter   5 | Accuracy: 0.615 | Bias: 0.003
Iter  10 | Accuracy: 0.331 | Bias: 0.041
Iter  15 | Accuracy: 0.713 | Bias: 0.030
Iter  20 | Accuracy: 0.753 | Bias: 0.055
Iter  25 | Accuracy: 0.715 | Bias: 0.058
Iter  30 | Accuracy: 0.617 | Bias: 0.073
Iter  35 | Accuracy: 0.617 | Bias: 0.098
Iter  40 | Accuracy: 0.617 | Bias: 0.101
Iter  45 | Accuracy: 0.617 | Bias: 0.133
Iter  50 | Accuracy: 0.617 | Bias: 0.129
Iter  55 | Accuracy: 0.617 | Bias: 0.141
Iter  60 | Accuracy: 0.617 | Bias: 0.156
Iter  65 | Accuracy: 0.617 | Bias: 0.155
